In [136]:
"""
copyright (c) 2012-2020 PEQNP. all rights reserved. contact@peqnp.science
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

In [ ]:
# https://github.com/maxtuno/PEQNP
# pip install PEQNP --upgrade
from peqnp import *
import pandas as pd

In [137]:
# https://www.kaggle.com/karangadiya/fifa19
data = pd.read_csv('fifa2019.csv')
data.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


In [138]:
size = 100 # change acording your RAM
matrix = data[['Overall', 'Potential', 'Special', 'International Reputation', 'Skill Moves', 'Value']][:size]
matrix['Value'] = [float(v.strip('€').strip('M')) for v in  matrix['Value'].values]
matrix.head()

,Overall,Potential,Special,International Reputation,Skill Moves,Value
0,94,94,2202,5.0,4.0,110.5
1,94,94,2228,5.0,5.0,77.0
2,92,93,2143,5.0,5.0,118.5
3,91,93,1471,4.0,1.0,72.0
4,91,92,2281,4.0,4.0,102.0


In [139]:
matrix = matrix.values
bits = int(sum(matrix.flatten())).bit_length()

In [140]:
overall = 0
potential = 0
special = 0
skill_moves = 0
reputation = 0
value = 2 ** bits - 1
while True:
    engine(bits)
    x = integer(bits=matrix.size)
    assert sum(switch(x, i, neg=True) for i in range(size)) == 11
    assert sum(switch(x, i, neg=True) * matrix[i][0] for i in range(size)) >= overall
    assert sum(switch(x, i, neg=True) * matrix[i][1] for i in range(size)) >= potential
    assert sum(switch(x, i, neg=True) * matrix[i][2] for i in range(size)) >= reputation
    assert sum(switch(x, i, neg=True) * matrix[i][3] for i in range(size)) >= special
    assert sum(switch(x, i, neg=True) * matrix[i][4] for i in range(size)) >= skill_moves
    assert sum(switch(x, i, neg=True) * matrix[i][5] for i in range(size)) < value
    if satisfy(turbo=True):
        overall = sum(matrix[i][0] for i in range(size) if x.binary[i])
        potential = sum(matrix[i][1] for i in range(size) if x.binary[i])        
        reputation = sum(matrix[i][2] for i in range(size) if x.binary[i])
        special = sum(matrix[i][3] for i in range(size) if x.binary[i])
        skill_moves = sum(matrix[i][4] for i in range(size) if x.binary[i])
        value = sum([matrix[i][5] for i in range(size) if x.binary[i]])
        print(potential, reputation, special, skill_moves, reputation, skill_moves, value)
        players = [data.values[i][1] for i in range(size) if x.binary[i]]
    else:
        break
print('OPTIMUM FOUND')

1001.0 22840.0 43.0 36.0 22840.0 36.0 791.5
1004.0 22942.0 46.0 38.0 22942.0 38.0 784.0
1004.0 23246.0 46.0 39.0 23246.0 39.0 778.5
1006.0 23459.0 46.0 39.0 23459.0 39.0 775.5
OPTIMUM FOUND


In [141]:
all_stars = data[data['ID'].isin(players)]

In [142]:
all_stars

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
6,6,177003,L. Modrić,32,https://cdn.sofifa.org/players/4/19/177003.png,Croatia,https://cdn.sofifa.org/flags/10.png,91,91,Real Madrid,...,84.0,60.0,76.0,73.0,13.0,9.0,7.0,14.0,9.0,€137.4M
7,7,176580,L. Suárez,31,https://cdn.sofifa.org/players/4/19/176580.png,Uruguay,https://cdn.sofifa.org/flags/60.png,91,91,FC Barcelona,...,85.0,62.0,45.0,38.0,27.0,25.0,31.0,33.0,37.0,€164M
8,8,155862,Sergio Ramos,32,https://cdn.sofifa.org/players/4/19/155862.png,Spain,https://cdn.sofifa.org/flags/45.png,91,91,Real Madrid,...,82.0,87.0,92.0,91.0,11.0,8.0,9.0,7.0,11.0,€104.6M
12,12,182493,D. Godín,32,https://cdn.sofifa.org/players/4/19/182493.png,Uruguay,https://cdn.sofifa.org/flags/60.png,90,90,Atlético Madrid,...,82.0,90.0,89.0,89.0,6.0,8.0,15.0,5.0,15.0,€90.2M
13,13,168542,David Silva,32,https://cdn.sofifa.org/players/4/19/168542.png,Spain,https://cdn.sofifa.org/flags/45.png,90,90,Manchester City,...,93.0,59.0,53.0,29.0,6.0,15.0,7.0,6.0,12.0,€111M
15,15,211110,P. Dybala,24,https://cdn.sofifa.org/players/4/19/211110.png,Argentina,https://cdn.sofifa.org/flags/52.png,89,94,Juventus,...,84.0,23.0,20.0,20.0,5.0,4.0,4.0,5.0,8.0,€153.5M
20,20,189511,Sergio Busquets,29,https://cdn.sofifa.org/players/4/19/189511.png,Spain,https://cdn.sofifa.org/flags/45.png,89,89,FC Barcelona,...,90.0,90.0,86.0,80.0,5.0,8.0,13.0,9.0,13.0,€105.6M
